In [ ]:
import csv
import os
import sys
import numpy as np
from nlp_util import *
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
import pickle

import numpy as np
import codecs
import operator

In [ ]:
#define IEMOCAP path
IEMOCAP_PATH = ''

# define functions for file management
def file_search(dirname, ret, list_avoid_dir=[]):
    filenames = os.listdir(dirname)  
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        if os.path.isdir(full_filename) :
            if full_filename.split('/')[-1] in list_avoid_dir:
                continue
            else:
                file_search(full_filename, ret, list_avoid_dir)           
        else:
            ret.append( full_filename )          

def find_encoding(filename):
    rawdata = open(filename, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']    
    return charenc

In [ ]:
# define functions for extracting transcripttion
def extract_trans( list_in_file, out_file ) :
    lines = []  
    for in_file in list_in_file:
        cnt = 0
        with open(in_file, 'r') as f:
            lines = f.readlines()
        with open(out_file, 'a') as f:
            csv_writer = csv.writer( f )
            lines = sorted(lines)                  
            for line in lines:
                name = line.split(':')[0].split(' ')[0].strip()
                # remove unwanted cases (noises, umatched pair in label)
                if name[:3] != 'Ses':           
                    continue
                elif name[-3:-1] == 'XX':       
                    continue
                trans = line.split(':')[1].strip()
                cnt += 1
                csv_writer.writerow([name, trans])

In [ ]:
# Extract transcriptions
out_file = 'processed_transcription.csv'
os.system('rm' + out_file)

list_files = []

for x in xrange(5):
    sess_name = 'Session' + str(x+1)
    path = IEMOCAP_PATH + sess_name + '/dialog/transcriptions/'
    # sample: Ses01F_impro01_F000 [006.2901-008.2357]: Excuse me.
    file_search(path, list_files)
    list_files = sorted(list_files)
    print sess_name + ", #sum files: " + str(len(list_files))

extract_trans(list_files, out_file)

In [ ]:
# define functions for labeling
def find_category(lines):
    is_target = True  
    id = ''
    c_label = ''
    list_ret = []
    for line in lines:
        if is_target == True:
            try:
                id  = line.split('\t')[1].strip()  
                c_label  = line.split('\t')[2].strip()  
                if not category.has_key(c_label):
                    print "No key: " + c_label
                    sys.exit()
                list_ret.append( [id, c_label] )
                is_target = False
            except:
                print "ERROR " + line
                sys.exit()
        else:
            if line == '\n':
                is_target = True
    return list_ret

def extract_labels( list_in_file, out_file ) :
    id = ''
    lines = []
    list_ret = []
    for in_file in list_in_file:
        with open(in_file, 'r') as f:
            lines = f.readlines()
            lines = lines[2:]                           
            list_ret = find_category(lines)
        list_ret = sorted(list_ret)                   
        with open(out_file, 'a') as f:
            csv_writer = csv.writer( f )
            csv_writer.writerows( list_ret )

In [ ]:
# Extract label
out_file = 'label.csv'
os.system('rm' + out_file)

category = {'ang': 0, 'hap': 1, 'sad': 2, 'neu': 3, 'fru': 4, 'exc': 5, 'fea': 6,  'sur': 7, 'dis': 8, 'oth': 9, 'xxx': 10}

list_files = []
list_avoid_dir = ['Attribute', 'Categorical', 'Self-evaluation']

for x in xrange(5):
    sess_name = 'Session' + str(x+1)
    path = IEMOCAP_PATH + sess_name + '/dialog/EmoEvaluation/'
    file_search(path, list_files, list_avoid_dir)
    list_files = sorted(list_files)
    print sess_name + ", #sum files: " + str(len(list_files))

extract_labels(list_files, out_file)

lines = []
with open(out_file, 'r') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]
    
new_out_file = 'new_label.csv'
os.system('rm' + new_out_file)
with open(new_out_file, 'a') as f:
    csv_writer = csv.writer( f )
    for line in lines:
        if line[1] == 'ang': csv_writer.writerow (['0'])
        elif line[1] == 'hap': csv_writer.writerow (['1'])
        elif line[1] == 'exc': csv_writer.writerow (['1'])
        elif line[1] == 'sad': csv_writer.writerow (['2'])
        elif line[1] == 'neu': csv_writer.writerow (['3'])
        else : csv_writer.writerow (['-1'])
        

In [ ]:
lines = []
with open(new_out_file, 'r') as f:
    csv_reader = csv.reader(f)
    lines = [x for x in csv_reader]

print len([x for x in lines if x==['0']])
print len([x for x in lines if x==['1']])
print len([x for x in lines if x==['2']])
print len([x for x in lines if x==['3']])

In [ ]:
# Verify 
in_file1 = 'label.csv'
in_file2 = 'processed_transcription.csv'
in_file3 = 'new_label.csv'

label = []
trans = []
with open( in_file1, 'r') as f:
    csv_reader = csv.reader( f )
    label = [x for x in csv_reader]   
label_id = [x[0] for x in label]
label_var = [x[1] for x in label]

with open( in_file2, 'r') as f:
    csv_reader = csv.reader( f )
    tran = [x for x in csv_reader]   
tran_id = [x[0] for x in tran]

with open( in_file3, 'r') as f:
    csv_reader = csv.reader( f )
    new_label = [x for x in csv_reader]   
new_label_id = [x[0] for x in new_label]

chk_miss = False
for l, t in zip(label_id, tran_id):
    if l != t:
        print 'ERROR'
        chk_miss = True

if not(chk_miss): print("NO INCORRECT DATA")
print("COUNT: ", len(label_id), len(tran_id), len(new_label_id))

count = np.zeros (len(category), dtype=np.int)
for l in label_var:
    count[category[l]] += 1

for k in category:
    print (k+' : '+str(count[category[k]]))


In [ ]:
## Create voca from transcription

In [ ]:
# define functions for NLP
def apply_mincut(dic, min_freq):
    print 'apply minCut and re-generate minCutDic'
    mincut_dic = dict(filter(lambda (a, b) : b > min_freq, dic.items()))
    print 'minFreq = ' + str(min_freq)
    print 'original dic size = ' + str(len(dic))
    print 'original dic word freq = ' + str(sum(dic.values()))
    print 'minCut dic size = ' + str(len(mincut_dic))
    print 'minCut dic word freq = ' + str(sum(mincut_dic.values()))
    coverage = sum(mincut_dic.values()) / float(sum(dic.values()))
    print 'coverage = ' + str(coverage)
    return mincut_dic

# value <-> key
def create_invert_dic( dic ) :
    inv_dic = {}
    for key in dic.keys() :
        inv_dic[ dic[key] ] = key
    
    return inv_dic

# print sentences
def index_to_sentence( inv_dic, list_index ) :
    print [ inv_dic[ x ] for x in list_index ]

In [ ]:
# Create voca and check
lines = []
with open('processed_transcription.csv') as f:
    read = csv.reader(f)
    lines = [ x[1] for x in read]
    
token_lines = [word_tokenize(x) for x in lines]
#word_tokenize: param text:	text to split into sentences
#e.g.: ['Excuse', 'me', '.']
token_lines_lower = [ [t.lower() for t in x] for x in token_lines]
sent_len = [len(x) for x in token_lines]

print 'max : ' + str(np.max(sent_len))
print 'min : ' + str(np.min(sent_len))
print 'mean : ' + str(np.mean(sent_len))
print 'std : ' + str(np.std(sent_len))

dic_count = {}
for tokens in token_lines_lower:
    for token in tokens:
        token = token.lower()
        if token in dic_count :
            dic_count[token] += 1
        else:
            dic_count[token] = 1
print 'dic size : ' + str(len(dic_count))

# dic_count = apply_mincut(dic_count, 1)
# print 'dic size : ' + str(len(dic_count))



In [ ]:
dic = {}
dic['_PAD_'] = 0
dic['_UNK_'] = 0

for word in dic_count.keys():
    dic[word] = len(dic)    
print len(dic)
print dic['_PAD_']
print dic['_UNK_']

with open('dic.pkl', 'w') as f:
    pickle.dump( dic, f )
    
# convert to index
index_lines = [ [ dic[t] for t in x ] for x in token_lines_lower ]

len_max = np.max(sent_len)
np_trans = np.zeros( [len(index_lines), len_max], dtype=np.int)
for i in range(len(index_lines)):
    if len( index_lines[i] ) >= len_max:
        np_trans[i][:] = index_lines[i][:len_max]
    else:
        np_trans[i][:len(index_lines[i])] = index_lines[i][:]

print(np.shape(np_trans))
np.save('processed_transcription.npy', np_trans)
print("DONE")

In [ ]:
## Create word embedding (GloVe)

In [ ]:
# define functions for creating word embedding
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = codecs.open(gloveFile,'r')
    
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

    f.close()
    
def cal_coverage(voca, glove):
    cnt = 0
    for token in voca.keys():
        if glove.has_key( token ):
            ;
        else:
            cnt = cnt + 1  

    print '# missing token : ' + str(cnt)
    print 'coverage : ' + str( 1 - ( cnt/ float(len(voca)) ) ) 
    
def create_glove_embedding(voca, glove):

    # sorting voca dic by value
    sorted_voca = sorted(voca.items(), key=operator.itemgetter(1))
    
    list_glove_voca = []

    cnt = 0

    for token, value in sorted_voca:

        if glove.has_key( token ):
            list_glove_voca.append( glove[token] )
        else:
            if token == '_PAD_':
                print 'add PAD as 0s'
                list_glove_voca.append( np.zeros(300) )  
            else:
                list_glove_voca.append( np.random.uniform(-0.25, 0.25, 300).tolist() )
                cnt = cnt + 1  

    print 'coverage : ' + str( 1 - ( cnt/ float(len(voca)) ) )
    return list_glove_voca

In [ ]:
# load GloVe model
print 'loading started'
# Ref: download 'Common Crawl' from 'https://nlp.stanford.edu/projects/glove/'
glove = loadGloveModel('E:\MyGithub\GloVe_RAW_DATA\glove.840B.300d.txt')
print 'loading completed'


In [ ]:
# create & store word embedding
dic = {}
with open('dic.pkl') as f:
    dic = pickle.load(f)
print 'total dic size : ' + str(len(dic))

cal_coverage(dic, glove)

list_glove_voca = create_glove_embedding(dic, glove)
len(list_glove_voca)

np_glove = np.asarray(list_glove_voca, dtype=np.float32)
print np.shape(np_glove)
np.save('word_embedding.npy', np_glove)